<a href="https://colab.research.google.com/github/adithyaGHegde/Data-Science-ML-AI/blob/main/Youtube_Comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import urllib.request
import re
import os
# from dotenv import load_dotenv
# load_dotenv()
API_KEY = ""


In [2]:
from googleapiclient.discovery import build
# creating youtube resource object
youtube = build("youtube", "v3", developerKey = API_KEY)

In [3]:
def checkparameter(dict,key):
    if key in dict:
        return dict[key]
    else:
        return None

In [4]:
def get_comment_data(video_id):

    replies = []
    count = 0

    list_of_comments = []


    if "commentCount" in (youtube.videos().list(part="statistics",id=video_id).execute())['items'][0]["statistics"]:
      video_response=youtube.commentThreads().list(part='snippet,replies',videoId=video_id).execute()
    else:
      return(list_of_comments)
    
    count = 0

    while video_response:


        for item in video_response['items']:
            
            comment = checkparameter(item['snippet']['topLevelComment']['snippet'],'textOriginal')
            comment_like_count = checkparameter(item['snippet']['topLevelComment']['snippet'],'likeCount')

            replycount = checkparameter(item['snippet'],'totalReplyCount')

            replies = []
            replies_like_count = []

            if replycount>0:

                item_replies = checkparameter(item,'replies')

                if item_replies==None:
                  return

                for reply in item_replies['comments']:

                    reply_like_count = checkparameter(reply['snippet'],'likeCount')
                    reply = checkparameter(reply['snippet'],'textOriginal')
                      
                    replies.append(reply)
                    replies_like_count.append(reply_like_count)
  
            comment_thread = [comment,comment_like_count,replies,replies_like_count]
            list_of_comments.append(comment_thread)

            replies = []
            replies_like_count = []
  
        if ('nextPageToken' in video_response):
            video_response = youtube.commentThreads().list(part  = 'snippet,replies',videoId = video_id).execute()
            count += 1
        else:
            break
    return(list_of_comments)

In [5]:
def makeDataframe(video_ids):

    final_df = pd.DataFrame(columns=["Video Id","Comment","Likes","Replies","Likes on replies"])

    for video_id in video_ids:
        comments_df = pd.DataFrame(get_comment_data(video_id),columns=["Comment","Likes","Replies","Likes on replies"])
        comments_df.insert(0,"Video Id",video_id)
        final_df = pd.concat([final_df,comments_df])

    return(final_df)


In [6]:
def queryResult():
    keywords = (input("Enter keywords:").split())
    search_keyword = '+'.join(keywords)

    html = urllib.request.urlopen("https://www.youtube.com/results?search_query=" + search_keyword)
    video_ids = re.findall(r"watch\?v=(\S{11})", html.read().decode())
    list_set = set(video_ids)
    video_ids = list(list_set)

    return(video_ids)

In [7]:
def main():
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)

    video_ids = queryResult()

    comments_df = makeDataframe(video_ids)
    
    comments_df.to_csv("out.csv",encoding ='utf-8')
    print(comments_df.head(25))

In [8]:
def checkparameter(dict,key):
    if key in dict:
        return dict[key]
    else:
        return None

In [ ]:
if __name__ == "__main__":
    main()

Enter keywords:iiit dharwad


In [ ]:
from google.colab import files
files.download("out.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>